In [ ]:
# -*- coding: utf-8 -*-
%tensorflow_version 1.x
!pip install keras==2.2.5
!pip install pyvi

import numpy as np
from numpy import random
import os, pickle, re, keras, sklearn, string
from keras.callbacks import *
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from pyvi import ViTokenizer, ViPosTagger
from keras.layers import *
from keras.models import Model
from keras import optimizers
import gensim, operator, json
import pandas as pd
from sklearn.metrics import *
import keras.backend as K
from keras.models import *
from keras import initializers, regularizers
from keras import optimizers
from keras.engine.topology import Layer
from keras import constraints


In [ ]:
# Đọc đường dẫn data lưu trên google drive
from google.colab import drive
drive.mount('/content/gdrive')

path_train ='/content/gdrive/My Drive/Project/Emotion Text Classification/data/train_nor_811.csv'
path_valid ='/content/gdrive/My Drive/Project/Emotion Text Classification/data/valid_nor_811.csv'
path_test ='/content/gdrive/My Drive/Project/Emotion Text Classification/data/test_nor_811.csv'
path_stopword = '/content/gdrive/My Drive/Emotion Text Classification/data/stopwords.txt'


In [ ]:
# Hàm đọc thư viện pretrain word embedding.
# Bộ pretrain word embedding được huấn luyện trên báo mới, news có 300 chiều

path_embedding= '/content/gdrive/My Drive/Project/Emotion Text Classification/embedding/baomoi.vn.model.bin'

import io
from gensim.models import KeyedVectors

word_embedding = KeyedVectors.load_word2vec_format(path_embedding, binary=True)

# Ví dụ lấy vector của 1 từ trong pretrain word embedding
EMBEDDING_DIM = word_embedding['yêu'].shape[0]
print("Chieu du lieu embedding: ",EMBEDDING_DIM)
# Vector của từ yêu trong bộ pretrained word embedding.
print(word_embedding['yêu'])


In [ ]:
# Các hàm tiền xử lý 
def tokenizer(text):
    token = ViTokenizer.tokenize(text)
    return token

# Các bước tiền xử lý dữ liệu tiếng Việt
# Xóa các icon trong văn bản

def deleteIcon(text):
    text = text.lower()
    s = ''
    pattern = r"[a-zA-ZaăâbcdđeêghiklmnoôơpqrstuưvxyàằầbcdđèềghìklmnòồờpqrstùừvxỳáắấbcdđéếghíklmnóốớpqrstúứvxýảẳẩbcdđẻểghỉklmnỏổởpqrstủửvxỷạặậbcdđẹệghịklmnọộợpqrstụựvxỵãẵẫbcdđẽễghĩklmnõỗỡpqrstũữvxỹAĂÂBCDĐEÊGHIKLMNOÔƠPQRSTUƯVXYÀẰẦBCDĐÈỀGHÌKLMNÒỒỜPQRSTÙỪVXỲÁẮẤBCDĐÉẾGHÍKLMNÓỐỚPQRSTÚỨVXÝẠẶẬBCDĐẸỆGHỊKLMNỌỘỢPQRSTỤỰVXỴẢẲẨBCDĐẺỂGHỈKLMNỎỔỞPQRSTỦỬVXỶÃẴẪBCDĐẼỄGHĨKLMNÕỖỠPQRSTŨỮVXỸ,._]"
    for char in text:
        if char !=' ':
            if len(re.findall(pattern, char)) != 0:
                s+=char
            elif char == '_':
                s+=char
        else:
            s+=char
    s = re.sub('\\s+',' ',s)
    return s.strip()

# Hàm xử lý chính tiền xử lý
def clean_doc(doc):
    doc = tokenizer(doc)
    # xóa tất cả dấu câu (!,?..) trong câu
    for punc in string.punctuation:
        if punc != "_":
            doc = doc.replace(punc,' ')
    doc = deleteIcon(doc)
    
    # Xóa các số 
    doc = re.sub(r"[0-9]+", " num ", doc)
    # Đưa về chữ thường
    doc = doc.lower()
    # Xóa nhiều khoảng trắng
    doc = re.sub('\\s+',' ',doc)
    return doc

In [ ]:
# hàm đọc dữ liệu trên 3 tập train dev test và chạy tiền xử lý 
train_data = pd.read_csv(path_train)
valid_data = pd.read_csv(path_valid)
test_data = pd.read_csv(path_test)

X_train = train_data["Sentence"].apply(lambda x : clean_doc(x))
y_train = train_data["Emotion"]

X_val = valid_data["Sentence"].apply(lambda x : clean_doc(x))
y_val = valid_data["Emotion"]

X_test = test_data["Sentence"].apply(lambda x : clean_doc(x))
y_test = test_data["Emotion"]

In [ ]:
# Hiển thị kết quả số lượng tập train và tập test
print(len(X_train),len(y_train))
print(len(X_val),len(y_val))
print(len(X_test),len(y_test))


In [ ]:
classes = ['Anger','Disgust','Enjoyment','Fear','Other','Sadness','Surprise']
def to_category_vector(label):
    vector = np.zeros(len(classes)).astype(np.float64)
    index = classes.index(label)
    vector[index] = 1.0
    return vector

In [ ]:
# Chuyển nhãn thành số trong tập train và tập test

y_train_encode = []
for label in y_train:
    y_train_encode.append(to_category_vector(label))


y_val_encode = []
for label in y_val:
    y_val_encode.append(to_category_vector(label))

print(classes)
print(y_train_encode[0])
print(y_train[0])

In [ ]:
# Tất cả các từ vựng trong tập X_train nó sẽ tạo thành từ điển
# mỗi vector của từ đầu vào, nó sẽ chuyển thành 1 vector với số chiều cố định và mỗi các từ vựng sẽ thay bằng chỉ số index của nó trong tập từ điển
# Số chiều vector mỗi đầu vào ta sẽ lấy câu dài nhất là chiều vector đâu vào và các caua ngắn hơn sẽ tự động thêm giá trị 0 phía sau
xLengths = [len(x.split(' ')) for x in X_train]
h = sorted(xLengths)  #sorted lengths
maxLength =h[len(h)-1]
print("Giá trị độ dài câu dài nhất: ",maxLength)
input_tokenizer = Tokenizer(filters="",oov_token="UNK")
input_tokenizer.fit_on_texts(X_train)
input_vocab_size = len(input_tokenizer.word_index) + 1
word_index = input_tokenizer.word_index
print("input_vocab_size:",input_vocab_size)
X_train_encode = np.array(pad_sequences(input_tokenizer.texts_to_sequences(X_train), maxlen=maxLength,padding="post"))

# Ví dụ đầu vào mô hình LSTM
print("Câu đầu vào : ", X_train[0])
print("Câu đầu vào sau khi encode : ",X_train_encode[0])

X_val_encode = np.array(pad_sequences(input_tokenizer.texts_to_sequences(X_val), maxlen=maxLength,padding="post"))



In [ ]:
# Hàm lấy vector của từ vựng trong pre-trained word embedding 
def generate_embedding(word_index, model_embedding,EMBEDDING_DIM):
    count6 = 0
    countNot6 = 0
    #embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM)) 
    embedding_matrix = np.asarray([np.random.uniform(-0.01,0.01,EMBEDDING_DIM) for _ in range((len(word_index) + 1))])
    list_oov = []
    word_is_trained = []
    for word, i in word_index.items():
        try:
            embedding_vector = model_embedding[word]
            word_is_trained.append(word)
        except:
            continue
        if embedding_vector is not None:
            count6 +=1
            embedding_matrix[i] = embedding_vector
    
    print('Number of words in pre-train embedding: ' + str(count6))
    print('Number of words not in pre-train embedding: ' + str(countNot6))
    return embedding_matrix,word_is_trained

In [ ]:
embedding_matrix,word_is_trained = generate_embedding(word_index,word_embedding,EMBEDDING_DIM)
print(word_is_trained)


In [ ]:
"""
# ATTENTION LAYER
Cite these works 
1. Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
"Hierarchical Attention Networks for Document Classification"
accepted in NAACL 2016
"""
def dot_product(x, kernel):
	if K.backend() == 'tensorflow':
		return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
	else:
		return K.dot(x, kernel)

class AttentionWithContext(Layer):
	def __init__(self,
				 W_regularizer=None, u_regularizer=None, b_regularizer=None,
				 W_constraint=None, u_constraint=None, b_constraint=None,
				 bias=True, **kwargs):

		self.supports_masking = True
		self.init = initializers.get('glorot_uniform')

		self.W_regularizer = regularizers.get(W_regularizer)
		self.u_regularizer = regularizers.get(u_regularizer)
		self.b_regularizer = regularizers.get(b_regularizer)

		self.W_constraint = constraints.get(W_constraint)
		self.u_constraint = constraints.get(u_constraint)
		self.b_constraint = constraints.get(b_constraint)

		self.bias = bias
		super(AttentionWithContext, self).__init__(**kwargs)

	def build(self, input_shape):
		assert len(input_shape) == 3

		self.W = self.add_weight((input_shape[-1], input_shape[-1],),
								 initializer=self.init,
								 name='{}_W'.format(self.name),
								 regularizer=self.W_regularizer,
								 constraint=self.W_constraint)
		if self.bias:
			self.b = self.add_weight((input_shape[-1],),
									 initializer='zero',
									 name='{}_b'.format(self.name),
									 regularizer=self.b_regularizer,
									 constraint=self.b_constraint)

		self.u = self.add_weight((input_shape[-1],),
								 initializer=self.init,
								 name='{}_u'.format(self.name),
								 regularizer=self.u_regularizer,
								 constraint=self.u_constraint)

		super(AttentionWithContext, self).build(input_shape)

	def compute_mask(self, input, input_mask=None):
		# do not pass the mask to the next layers
		return None

	def call(self, x, mask=None):
		uit = dot_product(x, self.W)

		if self.bias:
			uit += self.b

		uit = K.tanh(uit)
		ait = dot_product(uit, self.u)

		a = K.exp(ait)

		if mask is not None:
			a *= K.cast(mask, K.floatx())
		a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

		a = K.expand_dims(a)
		weighted_input = x * a
		
		return weighted_input

	def compute_output_shape(self, input_shape):
		return input_shape[0], input_shape[1], input_shape[2]
	
class Addition(Layer):
	def __init__(self, **kwargs):
		super(Addition, self).__init__(**kwargs)

	def build(self, input_shape):
		self.output_dim = input_shape[-1]
		super(Addition, self).build(input_shape)

	def call(self, x):
		return K.sum(x, axis=1)

	def compute_output_shape(self, input_shape):
		return (input_shape[0], self.output_dim)

In [ ]:
# Khởi tạo mô hình LSTM. 
filter_nums = 256 # best 128
def build_model():
        inputs  = Input(shape=(maxLength, ), dtype='float64', name='inputs')    
        embedding_layer = Embedding(input_vocab_size,EMBEDDING_DIM,weights=[embedding_matrix], input_length=maxLength, trainable=True,name = 'word_emb')(inputs)
        embedding_layer = SpatialDropout1D(0.75)(embedding_layer)
                
              
        lstm_feature1 = CuDNNLSTM(filter_nums, return_sequences=True)(embedding_layer)

        att1 = AttentionWithContext()(lstm_feature1)
        att1 = Addition()(att1)

        fc1 = Dropout(0.5)(Dense(256, name = 'dense_1')(att1))
        output1 = Dense(len(classes),name="output1", activation='softmax')(fc1)

    
        # define optimizer

        model = Model(inputs=inputs, outputs=output1)
        tensorBoardCallback = TensorBoard(log_dir='./logs', write_graph=True)
        
        model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

        
        history = model.fit(X_train_encode, np.array(y_train_encode), validation_data = (X_val_encode,np.array(y_val_encode)) , batch_size=50, epochs=100,callbacks=[tensorBoardCallback])
        return model

model = build_model()

In [ ]:
X_test_encode = np.array(pad_sequences(input_tokenizer.texts_to_sequences(X_test), maxlen=maxLength,padding="post"))
test_length = len(X_test_encode)

y_predict = []
predicted = model.predict(X_test_encode)
for predict in predicted:
    index2, value = max(enumerate(predict), key=operator.itemgetter(1))
    y_predict.append(classes[index2])
    
print(y_predict[0])

In [ ]:
print(y_predict)

In [ ]:
# Dự đoán kết quả trên các độ đo
# precision = precision_score(y_test, y_predict, average='weighted')
# recall = recall_score(y_test, y_predict, average='weighted')
# f1score = f1_score(y_test, y_predict, average='micro')
accuracy = accuracy_score(y_test, y_predict)

# print("Kết quả mô hình LSTM + Attention layer")
print("Kết quả mô hình")
# print("Precision: ", precision)
# print("Recall: ", recall)
# print("F1-Score: ", f1score)
print("Accuracy: ", accuracy)

print(classification_report(y_test,y_predict))

In [ ]:
# Chương trình demo nhập vào 1 câu

demo = "Báo cáo thật là vui"
# chạy hàm tiền xử lý
demo_pre = clean_doc(demo)
# Đưa vào mô hình dự đoán
X_demo_encode = np.array(pad_sequences(input_tokenizer.texts_to_sequences([demo_pre]), maxlen=maxLength,padding="post"))
# Hiển thị kết quả encode câu đầu vào
# print(X_demo_encode)
predicted = model.predict(X_demo_encode)
index2, value = max(enumerate(predicted[0]), key=operator.itemgetter(1))
print("Kết quả dự đoán:", classes[index2])